In [9]:
#bemppcl
import numpy as np
import bempp.api
from bempp.api.operators.potential import laplace as lp
from bempp.api.operators.boundary import sparse, laplace, modified_helmholtz
vert = np.loadtxt ('vert.vert')
face = np.loadtxt ('face.face')-1
grid = bempp.api.Grid (np.transpose(vert), np.transpose(face))
space = bempp.api.function_space(grid,'DP', 0)
q = np.array([0.27, -0.70, 0.43])
x_q = np.array([[0.0190,0.00,0.6556],[-0.1197,0.00,-0.7372],[0.7370,0.00,-1.1390]])
ep_in = 4
ep_m = 4
k = 0.125
ep_s = 80.
K     = 0.5 * 4. * np.pi * 332.064

@bempp.api.real_callable
def charges_fun(x, n, domain_index, result):
    nrm = np.sqrt((x[0] - x_q[:, 0]) ** 2 + (x[1] - x_q[:, 1]) ** 2 + (x[2] - x_q[:, 2]) ** 2)
    aux = np.sum(q / nrm)
    result[0] = aux / (4 * np.pi * ep_in)
    
@bempp.api.real_callable    
def zero_i(x, n, domain_index, result):
    result[:] = 0

zero_grid_fun    = bempp.api.GridFunction(space, fun=zero_i     )    
charged_grid_fun = bempp.api.GridFunction(space, fun=charges_fun)
#print (charged_grid_fun.coefficients.real)
rhs = [charged_grid_fun, zero_grid_fun]

identity = sparse.identity(space, space, space)
slp_in   = laplace.single_layer(space, space, space)
dlp_in   = laplace.double_layer(space, space, space)

slp_out  = modified_helmholtz.single_layer(space, space, space, k)
dlp_out  = modified_helmholtz.double_layer(space, space, space, k)

blocked = bempp.api.BlockedOperator(2, 2)
blocked[0, 0] = 0.5*identity + dlp_in
blocked[0, 1] = -slp_in
blocked[1, 0] = 0.5*identity - dlp_out
blocked[1, 1] = (ep_m/ep_s)*slp_out

sol, info, it_count = bempp.api.linalg.gmres( blocked, rhs , return_iteration_count=True , tol=1e-5 ,
                                               use_strong_form=True)
U , dU = sol

#print (U.coefficients.real)
#print (dU.coefficients.real)

slp_in_O = lp.single_layer(space, x_q.transpose()) 
dlp_in_O = lp.double_layer(space, x_q.transpose())
U_R_O = slp_in_O * dU  -  dlp_in_O * U
S_trad     = K * np.sum(q * U_R_O).real

print (S_trad)

/home/elmobegazo/miniconda3/envs/bempp_prod_openmm/lib/python3.9/site-packages/scipy/sparse/linalg/dsolve/linsolve.py:318: SparseEfficiencyWarning: splu requires CSC matrix format
  warn('splu requires CSC matrix format', SparseEfficiencyWarning)


-5.23532736916225


In [45]:
a = np.array ([ 2.58808169e-03,  1.62113861e-03, -1.11696735e-03, -2.15266546e-03,
         -2.46334254e-03, -3.09950626e-03, -2.61145968e-03, -2.93678831e-03,
         -1.77125587e-03, -1.43403586e-03,  1.01500815e-03,  3.36363501e-03,
          1.14924191e-03,  1.18562558e-03,  1.05150484e-03,  1.06429670e-03,
          1.19507411e-03,  1.06544648e-03,  7.74953415e-04,  6.79938924e-04,
          8.65031437e-04,  7.44037648e-04,  8.36009817e-04,  4.80808091e-04,
         -2.58169946e-04, -5.89747381e-04,  5.18739817e-04,  1.04774819e-03,
          1.38958536e-03,  1.33637908e-03,  1.05145670e-03,  3.38541739e-04,
         -5.89747381e-04, -1.31735267e-05,  6.64722359e-04, -7.62338706e-04,
         -7.57556692e-05, -9.98110465e-04, -2.78101810e-03, -1.19882240e-03,
          4.81071294e-03,  7.92556116e-03, -9.52790243e-04, -2.67325708e-03,
         -3.57150524e-03, -4.00463556e-03, -3.93678667e-03, -3.23170577e-03])
b =np.array ([ 2.58808169e-03,  1.62113861e-03, -1.11696735e-03, -2.15266546e-03,
 -2.46334254e-03, -3.09950626e-03, -2.61145968e-03, -2.93678831e-03,
 -1.77125587e-03, -1.43403586e-03,  1.01500815e-03,  3.36363501e-03,
  1.14924191e-03,  1.18562558e-03,  1.05150484e-03,  1.06429670e-03,
  1.19507411e-03,  1.06544648e-03,  7.74953415e-04,  6.79938924e-04,
  8.65031437e-04,  7.44037648e-04,  8.36009817e-04,  4.80808091e-04,
 -2.58169946e-04, -5.89747381e-04,  5.18739817e-04,  1.04774819e-03,
  1.38958536e-03,  1.33637908e-03,  1.05145670e-03,  3.38541739e-04,
 -5.89747381e-04, -1.31735267e-05,  6.64722359e-04, -7.62338706e-04,
 -7.57556692e-05, -9.98110465e-04, -2.78101810e-03, -1.19882240e-03,
  4.81071294e-03,  7.92556116e-03, -9.52790243e-04, -2.67325708e-03,
 -3.57150524e-03, -4.00463556e-03, -3.93678667e-03, -3.23170577e-03])

In [46]:
print (a-b)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
